In [1]:
import re
import glob
import os
import time 
import datetime
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from Config import params_config

In [2]:
parameters = params_config.parameters
parameters

{'TARGET_URL': 'https://www.keibalab.jp/db/race/',
 'START_DATE': '19860105',
 'DRIVER_DIR': './chromedriver',
 'PAGE_LOAD_TIMEOUT': 10}

### 過去の競馬結果をスクレイピングするmain関数処理
- 1986/01/05 まで情報あり
- 土曜と日曜（レースがない日もある）だけ記録があるので、そこだけスクレイピングする

In [3]:
def get_today_datetime():
    return datetime.date.today()

In [4]:
def get_latest_holiday_list(target_datetime):
    while True:
        if target_datetime.weekday() == 5:
            saturday_datetime_str = target_datetime.strftime("%Y%m%d")
            target_datetime_plus_1 = target_datetime + datetime.timedelta(days=1)
            sunday_datetime_str = target_datetime_plus_1.strftime("%Y%m%d")
            return [saturday_datetime_str, sunday_datetime_str]
        else:
            target_datetime = target_datetime + datetime.timedelta(days=-1)

In [5]:
def scraping_race_result_until_start_date(parameters):
    # first process for scraping
    target_datetime =  get_today_datetime()  # type: datetime.date
    target_datetime_list =  get_latest_holiday_list(target_datetime)
    print(target_datetime_list)
    
    # scarping keibalab web info
    print('scraping the web site info in {TARGET_DATE}'.format(TARGET_DATE = target_datetime_list[0]))
    ## ENTERE MY CODE ##
    print('scraping the web site info in {TARGET_DATE}'.format(TARGET_DATE = target_datetime_list[1]))
    ## ENTERE MY CODE ##
    
    # after the second time process for scraping
    while True:
        target_datetime_str = target_datetime_list[0]  # type: str
        target_datetime = datetime.datetime.strptime(target_datetime_str, '%Y%m%d') - datetime.timedelta(days=1)  # type: datetime.date
        target_datetime_list =  get_latest_holiday_list(target_datetime)
        print(target_datetime_list)
        
        # scarping keibalab web info
        print('scraping the web site info in {TARGET_DATE}'.format(TARGET_DATE = target_datetime_list[0]))
        ## ENTERE MY CODE ##
        print('scraping the web site info in {TARGET_DATE}'.format(TARGET_DATE = target_datetime_list[1]))
        ## ENTERE MY CODE ##

        if parameters['START_DATE'] in target_datetime_list:
            break

In [ ]:
scraping_race_result_until_start_date(parameters)

In [6]:
def initialize_chrome_driver(parameters):
    driver = webdriver.Chrome(executable_path=parameters['DRIVER_DIR'])
    driver.set_page_load_timeout(parameters['PAGE_LOAD_TIMEOUT'])
    driver.maximize_window()
    
    return driver 

In [7]:
def close_chrome_window(driver):
    driver.quit()

In [8]:
def back_chrome_window(driver):
    driver.back()

In [9]:
driver = initialize_chrome_driver(parameters)

In [ ]:
# back_chrome_window(driver)
# close_chrome_window(driver)

## Class: KeibaLabScraper

- 例：20181229,1230の日はレースがないため、例外処理を行うようにしておく

In [10]:
# first process for scraping
target_datetime =  get_today_datetime()  # type: datetime.date
target_datetime_list =  get_latest_holiday_list(target_datetime)
target_datetime_list

['20190629', '20190630']

In [11]:
def make_web_driver_click_by(driver, parameters, xpath, verbose=True):
    try:
        WebDriverWait(driver, parameters['PAGE_LOAD_TIMEOUT']).until(EC.element_to_be_clickable((By.XPATH, xpath)))
        driver.find_element_by_xpath(xpath).click()
        if verbose:
            print('We can load the XPATH and now locate in:', driver.current_url)
            
    except Exception as e:
        print(e)

In [12]:
def load_target_url_page(driver, parameters, target_datetime_str):
    target_url = parameters['TARGET_URL'] + target_datetime_str
    
    try:
        driver.get(target_url)
        print('We can load the URL:', driver.current_url)
    except:
        print("The page load was time out")

In [13]:
target_datetime_str = target_datetime_list[0]
load_target_url_page(driver, parameters, target_datetime_str)

We can load the URL: https://www.keibalab.jp/db/race/20190629


In [14]:
def make_xpath_list_of_race_result_link():
    xpath_list_of_race_result_link = []
    
    xpath_race_info_div = '//*[@id="raceInfo"]/div[1]'
    race_table_header_list = driver.find_element_by_xpath(xpath_race_info_div).find_elements_by_tag_name('table')
    
    for table_header_idx in range(len(race_table_header_list)):
        xpath_table_of_race_result = xpath_race_info_div + '/table[{TABLE_IDX_IDX}]'.format(TABLE_IDX_IDX=table_header_idx+1)
        race_table_body_elem_list = driver.find_element_by_xpath(xpath_table_of_race_result).find_element_by_tag_name('tbody').find_elements_by_tag_name('tr')
        
        for table_tbody_idx in range(len(race_table_body_elem_list)):
            xpath_tdoby_of_table_race_result = xpath_table_of_race_result + '/tbody/tr[{TBODY_IDX}]/td[2]/a'.format(TBODY_IDX=table_tbody_idx+1)
            xpath_list_of_race_result_link.append(xpath_tdoby_of_table_race_result)
            
    return xpath_list_of_race_result_link

In [17]:
def get_race_id():
    return int(re.sub('\\D', '', driver.current_url.split(parameters['TARGET_URL'])[1]))

In [15]:
xpath_list_of_race_result_link = make_xpath_list_of_race_result_link()
xpath_list_of_race_result_link

['//*[@id="raceInfo"]/div[1]/table[1]/tbody/tr[1]/td[2]/a',
 '//*[@id="raceInfo"]/div[1]/table[1]/tbody/tr[2]/td[2]/a',
 '//*[@id="raceInfo"]/div[1]/table[1]/tbody/tr[3]/td[2]/a',
 '//*[@id="raceInfo"]/div[1]/table[1]/tbody/tr[4]/td[2]/a',
 '//*[@id="raceInfo"]/div[1]/table[1]/tbody/tr[5]/td[2]/a',
 '//*[@id="raceInfo"]/div[1]/table[1]/tbody/tr[6]/td[2]/a',
 '//*[@id="raceInfo"]/div[1]/table[1]/tbody/tr[7]/td[2]/a',
 '//*[@id="raceInfo"]/div[1]/table[1]/tbody/tr[8]/td[2]/a',
 '//*[@id="raceInfo"]/div[1]/table[1]/tbody/tr[9]/td[2]/a',
 '//*[@id="raceInfo"]/div[1]/table[1]/tbody/tr[10]/td[2]/a',
 '//*[@id="raceInfo"]/div[1]/table[1]/tbody/tr[11]/td[2]/a',
 '//*[@id="raceInfo"]/div[1]/table[1]/tbody/tr[12]/td[2]/a',
 '//*[@id="raceInfo"]/div[1]/table[2]/tbody/tr[1]/td[2]/a',
 '//*[@id="raceInfo"]/div[1]/table[2]/tbody/tr[2]/td[2]/a',
 '//*[@id="raceInfo"]/div[1]/table[2]/tbody/tr[3]/td[2]/a',
 '//*[@id="raceInfo"]/div[1]/table[2]/tbody/tr[4]/td[2]/a',
 '//*[@id="raceInfo"]/div[1]/table[2]

In [16]:
xpath = xpath_list_of_race_result_link[0]
make_web_driver_click_by(driver, parameters, xpath, verbose=True)

We can load the XPATH and now locate in: https://www.keibalab.jp/db/race/201906290301/


In [20]:
race_id = get_race_id()
race_id

201906290301

### Get race list from schedule pages

In [ ]:
def get_race_info_from_schedule_page(driver):
    # find table tag info 
    race_schedule_block_elem = driver.find_element_by_class_name('race_schedule_block')
    race_schedule_block_elem_list = race_schedule_block_elem.find_element_by_tag_name('tbody').find_elements_by_tag_name('tr')
    
    # define table header
    race_schedule_block_header = race_schedule_block_elem_list[0].find_elements_by_tag_name('th')
    race_schedule_block_header = [race_schedule_block_header[i].text for i in range(len(race_schedule_block_header))]
    
    # make race info data frame
    race_schedule_block_info_list = []
#     for i in range(1, len(race_schedule_block_elem_list)):
    for i in range(1, len(race_schedule_block_elem_list)-120):
        race_schedule_block_body = race_schedule_block_elem_list[i].find_elements_by_tag_name('td')
        race_schedule_block_body = [race_schedule_block_body[i].text for i in range(len(race_schedule_block_body))]
        race_schedule_block_info_list.append(race_schedule_block_body)
    
    return pd.DataFrame(race_schedule_block_info_list, columns=race_schedule_block_header)

In [ ]:
race_schedule_block_df = get_race_info_from_schedule_page(driver)
race_schedule_block_df

### Click and get each race information

In [ ]:
def make_web_driver_click(driver, parameters, xpath, verbose=True):
    try:
        WebDriverWait(driver, parameters['PAGE_LOAD_TIMEOUT']).until(EC.element_to_be_clickable((By.XPATH, xpath)))
        driver.find_element_by_xpath(xpath).click()
        if verbose:
            print('We can load the XPATH and now locate in:', driver.current_url)
        
    except:
        print('The page load was time out')

In [ ]:
def click_into_top_page_of_each_race(driver, parameters, race_idx):
    xpath_to_each_race = '//*[@id="main"]/div/table/tbody/tr[{TR_IDX}]/td[2]/a'.format(TR_IDX=race_idx)
    make_web_driver_click(driver, parameters, xpath_to_each_race)

In [ ]:
def click_into_race_table_page_of_each_race(driver, parameters):
    xpath_to_each_race_table = '//*[@id="main"]/div[1]/div/div[2]/div[2]/div/ul[1]/li[2]/a/span'
    make_web_driver_click(driver, parameters, xpath_to_each_race_table, verbose=False) 
    
    xpath_to_each_race_table = '//*[@id="race_main"]/div[1]/ul[2]/li[1]/a'
    make_web_driver_click(driver, parameters, xpath_to_each_race_table) 

In [ ]:
race_idx = 9 # 2 to race_schedule_block_df.shape[0]+1
print('We get the information of race:', list(race_schedule_block_df.loc[race_idx-2, :]))

click_into_top_page_of_each_race(driver, parameters, race_idx)

In [ ]:
click_into_race_table_page_of_each_race(driver, parameters)

In [ ]:
def get_race_table(driver):
    xpath_to_race_table = '//*[@id="shutuba"]/diary_snap/table/tbody'
    race_table_elem = driver.find_element_by_xpath(xpath_to_race_table)

    race_table_header_elem = race_table_elem.find_element_by_xpath('tr').find_elements_by_xpath('th')
    race_table_header = [race_table_header_elem[i].text.replace('\n','') for i in range(len(race_table_header_elem))]

    race_table_body_elem = race_table_elem.find_elements_by_xpath('tr')[3:]
    race_table_body_list = []
    for i in range(len(race_table_body_elem)):
        race_table_body_elem_list = race_table_body_elem[i].find_elements_by_xpath('td')
        race_table_body = [race_table_body_elem_list[j].text.replace('\n','') for j in range(len(race_table_body_elem_list))]
        race_table_body_list.append(race_table_body[:-2])
        
    return pd.DataFrame(race_table_body_list, columns=race_table_header[:-1])

In [ ]:
get_race_table(driver)